<a href="https://colab.research.google.com/github/lishangli/DeepLearning/blob/master/LeNet_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim

n_epochs = 3
batch_size_train = 64
batch_size_test = 5000
learning_rate = 0.01
momentum = 0.5
log_interval = 100
random_seed = 1
torch.manual_seed(random_seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1.LeNet-5 模型定义

In [19]:


class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    self._conv1 = nn.Conv2d(1,6,5,1,2)
    self._pool1 = nn.MaxPool2d(2,stride=2)
    self._conv2 = nn.Conv2d(6,16,5)
    self._pool2 = nn.MaxPool2d(2,stride=2)
    self._c5 = nn.Linear(16*5*5,120)
    self._f6 = nn.Linear(120,84)
    self._output = nn.Linear(84,10)

  def forward(self,x):
    x = F.relu(self._conv1(x))
    x = self._pool1(x)
    x = F.relu(self._conv2(x))
    x = self._pool2(x)
    x = x.view(-1,16*5*5)
    x = F.relu(self._c5(x))
    x = F.relu(self._f6(x))
    x = self._output(x)
    return x



# 2.LeNet-5测试

In [17]:

input = torch.rand([32,1,32,32])
model = LeNet()
print(model)
output = model(input)
print(output)

LeNet(
  (_conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (_conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (_c5): Linear(in_features=400, out_features=120, bias=True)
  (_f6): Linear(in_features=120, out_features=84, bias=True)
  (_output): Linear(in_features=84, out_features=10, bias=True)
)
tensor([[ 0.0954,  0.0613, -0.0280, -0.0907,  0.0942,  0.0903, -0.0045,  0.0980,
          0.0219,  0.0516],
        [ 0.0971,  0.0616, -0.0252, -0.0932,  0.0897,  0.0892, -0.0020,  0.0968,
          0.0207,  0.0461],
        [ 0.0931,  0.0622, -0.0258, -0.0932,  0.0931,  0.0896, -0.0072,  0.0956,
          0.0219,  0.0476],
        [ 0.0975,  0.0644, -0.0230, -0.0911,  0.0894,  0.0887, -0.0088,  0.0921,
          0.0226,  0.0468],
        [ 0.0977,  0.0671, -0.0306, -0.0942,  0.0946,  0.0921, -0.0053,  

In [10]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data/', train=False, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_test, shuffle=True)

In [31]:
net = LeNet()
net.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.0001)
epoches = 5
for epoch in range(epoches):
    rloss = 0.0
    accuracy = 0.0
    for batch_idx,(image,label) in enumerate(train_loader):
        optimizer.zero_grad()
        #print(image.shape)
        image = image.to(device)
        label = label.to(device)
        outputs = net(image)
        loss = loss_function(outputs,label)
        loss.backward()
        optimizer.step()
        
        rloss += loss.item()
        if batch_idx % log_interval == 0:
          print('Train Epoch: {},[{}/{}，{}]，Loss:{}'.format(epoch,batch_idx * len(image),len(train_loader.dataset),(batch_idx * len(image)/len(train_loader.dataset)),loss.item()))
          rloss = 0.0
        
    with torch.no_grad():
      correct = 0.0
      total = 0
      for data in test_loader:
        image,label = data
        image,label = image.to(device),label.to(device)
        output = net(image)
        _,predicted = torch.max(output.data,1)
        total += label.size(0)
        correct += (predicted == label).sum()
      print('Epoch :{},{}'.format(epoch+1,float(correct/total)))
print('finish training')

#save_path = './Lenet.pth'
#torch.save(net.state_dict(), save_path)

Train Epoch: 0,[0/60000，0.0]，Loss:2.2947158813476562
Train Epoch: 0,[6400/60000，0.10666666666666667]，Loss:1.9002892971038818
Train Epoch: 0,[12800/60000，0.21333333333333335]，Loss:0.7822117209434509
Train Epoch: 0,[19200/60000，0.32]，Loss:0.34915533661842346
Train Epoch: 0,[25600/60000，0.4266666666666667]，Loss:0.31968656182289124
Train Epoch: 0,[32000/60000，0.5333333333333333]，Loss:0.21643362939357758
Train Epoch: 0,[38400/60000，0.64]，Loss:0.5314700603485107
Train Epoch: 0,[44800/60000，0.7466666666666667]，Loss:0.2795635163784027
Train Epoch: 0,[51200/60000，0.8533333333333334]，Loss:0.18216845393180847
Train Epoch: 0,[57600/60000，0.96]，Loss:0.29882410168647766
Epoch :1,0.9298999905586243
Train Epoch: 1,[0/60000，0.0]，Loss:0.22854264080524445
Train Epoch: 1,[6400/60000，0.10666666666666667]，Loss:0.19019050896167755
Train Epoch: 1,[12800/60000，0.21333333333333335]，Loss:0.23059532046318054
Train Epoch: 1,[19200/60000，0.32]，Loss:0.3186953067779541
Train Epoch: 1,[25600/60000，0.4266666666666667]，